## Dependencies

In [165]:
import datetime
import pandas as pd
import geopandas as gpd
from dataclasses import dataclass
import uuid

from shapely.geometry import MultiPolygon, Polygon

from farsiteutils import User, Run_File, Config_File, Input, FilePaths, Database

from multiprocessing import Pool

from ipyleaflet import Map, basemaps, basemap_to_tiles, ScaleControl, ZoomControl, LayersControl, WKTLayer, WidgetControl
from ipyleaflet import Circle
from ipyleaflet.leaflet import LayerException
from ipywidgets import Layout, FloatProgress, HBox, VBox, FloatText, Label
from ipywidgets import SelectionSlider

from functools import partial

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import os

import requests
import arrow

from dataclasses import dataclass

@dataclass
class Station:
    name: str
    lat: float
    lon: float
    temperature: list
    wind_speed: list
    wind_direction: list
    timestamp: list
    units: dict

In [233]:
def collect_wx_data():
    km_per_miles = 1.60934

    ignite = usr.db.gdfignition.to_crs(epsg=4326).loc[usr.igniteidx]
    compare = usr.db.gdfignition.to_crs(epsg=4326).loc[usr.compareidx]


    baseURL = "https://firemap.sdsc.edu/pylaski/stations/data?"

    ## Always start with the base of the URL

    selectionType='selection=withinRadius'

    # Choose the centroid of the ignition
    lon = ignite['geometry'].centroid.x
    lat = ignite['geometry'].centroid.y
    radius = 5*km_per_miles    # 2 mile radius

    ## Latitude and longitude of the White House according to Google
    selectionParameters = '&lat={}&lon={}&radius={}'.format(str(lat),str(lon), str(radius))

    observables_list = ['temperature', 'wind_speed', 'wind_direction']
    observables = ''
    for observable in observables_list:
        observables += f'&observable={observable}'

    fromdt = ignite['datetime'].tz_localize('US/Pacific')
    todt = compare['datetime'].tz_localize('US/Pacific')
    frm = arrow.Arrow.fromdatetime(fromdt.to_pydatetime())
    to = arrow.Arrow.fromdatetime(todt.to_pydatetime())

    frm = frm.shift(minutes=-10)
    to = to.shift(minutes=10)

    urlDateTime = '&from={}&to={}'.format(str(frm), str(to))

    urlPlot = baseURL + selectionType + selectionParameters + observables + urlDateTime
    print(urlPlot)

    r = requests.get(urlPlot)
    ## Request to GET information from the given URL (Our REST query we built)
    r_json = r.json()

    stations_lst = []
    for feature in r_json['features']:

        lon,lat = feature['geometry']['coordinates']

        name = feature['properties']['description']['name']
        wxid = feature['properties']['description']['id']
        provider = feature['properties']['description']['provider']
        wifire_uid = feature['properties']['description']['wifire_uid']

        timestamp = pd.to_datetime(feature['properties']['timestamp'])
        try:
            temperature = feature['properties']['temperature']
        except KeyError:
            temperature = []

        try:
            wind_speed = feature['properties']['wind_speed']
        except KeyError:
            wind_speed = []

        try:
            wind_direction = feature['properties']['wind_direction']
        except KeyError:
            wind_direction = []

        units = feature['properties']['units']

        stations_lst.append(Station(name, lat, lon, temperature, wind_speed, wind_direction, timestamp, units))
        
    return stations_lst

In [2]:
fp = FilePaths('/home/tcaglar/data/')
usr = User(fp)

Database interaction not yet implemented. Use pickle file for dataframes instead!
Setting up the interface


## Choose ignition


In [173]:
usr.m

Map(bottom=209252.0, center=[34.332096438353915, -119.00229713395525], controls=(AttributionControl(options=['…

{'windspeedlow': 0, 'windspeedhigh': 20, 'windspeeddelta': 5, 'winddirectionlow': 0, 'winddirectionhigh': 180, 'winddirectiondelta': 5, 'relhumid': 10, 'burntime': datetime.timedelta(0, 1800), 'burntimestep': datetime.timedelta(0, 1800), 'temperature': 60}
Choosing a perimeter from the database


In [246]:
usr.mainapi.run_farsite()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


KeyboardInterrupt: 

In [245]:
def collect_station_layers(stations_lst, ws_optimal):
    clayers_lst = []
    for station in stations_lst:
        try:
            ws = station.wind_speed[0]
            pdiff = abs(ws - ws_optimal)/ws_optimal

            blueint = int(pdiff*256)
            if blueint > 255:
                blueint = 255
            elif blueint < 0:
                blueint = 0
        except IndexError:
            blueint = 0
            
#         red = '{:02x}'.format(redint)
        blue = '{:02x}'.format(blueint)
        # Define circle
        clayer = Circle(radius=400, location=(station.lat, station.lon), stroke=False, fill_color='#{}0000'.format(blue), fill_opacity=1)
        
        # Add circle to circle list
        clayers_lst.append(clayer)
        
    return clayers_lst

for station in station_layers:
    usr.m.remove_layer(station)
    
geom_compare = usr.db.gdfignition.loc[usr.compareidx, 'geometry']
gdf = usr.mainapi.db.gdfsimulation

gdf['area'] = (gdf['geometry'].union(geom_compare).difference(gdf['geometry'].intersection(geom_compare))).area
idxmin = gdf['area'].idxmin()

ws_optimal = gdf.loc[idxmin, 'windspeed']
wd_optimal = gdf.loc[idxmin, 'winddirection']

stations_lst = collect_wx_data()

station_layers = collect_station_layers(stations_lst, ws_optimal)

for station in station_layers:
    usr.m.add_layer(station)    

# Color based on wind speed percent difference


https://firemap.sdsc.edu/pylaski/stations/data?selection=withinRadius&lat=34.31448975046042&lon=-119.04499757335797&radius=8.0467&observable=temperature&observable=wind_speed&observable=wind_direction&from=2019-10-31T19:48:22-07:00&to=2019-10-31T20:41:33-07:00


In [251]:
r_json['features'][0]['properties']

{'description': {'id': '134',
  'name': 'Camarillo, Camarillo Airport',
  'provider': 'Mesowest',
  'wifire_uid': 'mesowest_kcma'},
 'distanceFromLocation': {'value': 11.143267623372429, 'units': 'km'},
 'units': {'temperature': 'F', 'wind_speed': 'mps', 'wind_direction': 'd'},
 'timestamp': ['2019-10-31 19:00:00-0700',
  '2019-10-31 19:05:00-0700',
  '2019-10-31 19:10:00-0700',
  '2019-10-31 19:15:00-0700',
  '2019-10-31 19:20:00-0700',
  '2019-10-31 19:25:00-0700',
  '2019-10-31 19:30:00-0700',
  '2019-10-31 19:35:00-0700',
  '2019-10-31 19:40:00-0700',
  '2019-10-31 19:45:00-0700',
  '2019-10-31 19:50:00-0700',
  '2019-10-31 19:55:00-0700',
  '2019-10-31 20:00:00-0700',
  '2019-10-31 20:05:00-0700',
  '2019-10-31 20:10:00-0700',
  '2019-10-31 20:15:00-0700',
  '2019-10-31 20:20:00-0700',
  '2019-10-31 20:25:00-0700',
  '2019-10-31 20:30:00-0700',
  '2019-10-31 20:35:00-0700',
  '2019-10-31 20:40:00-0700',
  '2019-10-31 20:45:00-0700',
  '2019-10-31 20:50:00-0700',
  '2019-10-31 20:5

In [208]:
wd_optimal

95

In [209]:
stations_lst[0].wind_direction[0]

30.0

'0c'

37

In [140]:
frm

<Arrow [2019-10-31T19:58:22-07:00]>

In [13]:
wx_station_data = []
# Add weather stations within 10 miles


15

In [ ]:
usr.mainapi.runfile_lst[2].runpath

In [ ]:
def initiate_perimeters(gdf, newigniteid, newcompareid):
    add_mask_ignite = (gdf['objectid'] == newigniteid)
    add_mask_compare = (gdf['objectid'] == newcompareid)
    
    gdf.loc[add_mask_ignite, 'WKTLayerIgnite'].apply(lambda wlayer: m.add_layer(wlayer))
    gdf.loc[add_mask_compare, 'WKTLayerCompare'].apply(lambda wlayer: m.add_layer(wlayer))

    
def update_perimeters(gdf, vbox, oldigniteid, newigniteid, oldcompareid, newcompareid):    
    remove_mask_ignite = (gdf['objectid'] == oldigniteid)
    remove_mask_compare = (gdf['objectid'] == oldcompareid)
    add_mask_ignite = (gdf['objectid'] == newigniteid)
    add_mask_compare = (gdf['objectid'] == newcompareid)
    
    gdf.loc[remove_mask_ignite, 'WKTLayerIgnite'].apply(lambda wlayer: m.remove_layer(wlayer))
    gdf.loc[add_mask_ignite, 'WKTLayerIgnite'].apply(lambda wlayer: m.add_layer(wlayer))
    gdf.loc[remove_mask_compare, 'WKTLayerCompare'].apply(lambda wlayer: m.remove_layer(wlayer))
    gdf.loc[add_mask_compare, 'WKTLayerCompare'].apply(lambda wlayer: m.add_layer(wlayer))
    
    vbox.children[1].value = str(gdf.set_index('objectid').loc[newigniteid, 'datetime'])
    vbox.children[3].value = str(gdf.set_index('objectid').loc[newcompareid, 'datetime'])
    
    deltadt = gdf.set_index('objectid').loc[newcompareid, 'datetime'] - gdf.set_index('objectid').loc[newigniteid, 'datetime']
    vbox.children[4].value = str(int(deltadt.total_seconds()/60)) + ' minutes'
    
def observe_objectid_slider(m, gdf, vbox, event):
    if event['owner'].description == 'ignite':
        oldigniteid = event['old']
        newigniteid = event['new']
        oldcompareid = vbox.children[2].value
        newcompareid = vbox.children[2].value
    elif event['owner'].description == 'compare':
        oldigniteid = vbox.children[0].value
        newigniteid = vbox.children[0].value
        oldcompareid = event['old']
        newcompareid = event['new']
        
    update_perimeters(gdf, vbox, oldigniteid, newigniteid, oldcompareid, newcompareid)


gdf = usr.db.gdfignition.to_crs(epsg=4326)
gdf['WKTLayerIgnite'] = gdf.apply(lambda row: WKTLayer(wkt_string = row['geometry'].wkt), axis=1)
gdf['WKTLayerCompare'] = gdf.apply(lambda row: WKTLayer(wkt_string = row['geometry'].wkt), axis=1)

centerlon = gdf.iloc[0]['geometry'].centroid.x
centerlat = gdf.iloc[0]['geometry'].centroid.y

m = Map(
    basemap=basemaps.Esri.WorldTopoMap,
    center=(centerlat, centerlon),
    zoom=10,
    layout=Layout(height='800px', width='100%'),
    zoom_control=False
)



objectids = gdf['objectid'].unique()

igniteid_select = SelectionSlider(description='ignite', options=objectids)
ignitedt_label = Label(value=str(gdf.set_index('objectid').loc[igniteid_select.value, 'datetime']))

compareid_select = SelectionSlider(description='compare', options=objectids)
comparedt_label = Label(value=str(gdf.set_index('objectid').loc[compareid_select.value, 'datetime']))

deltadt = gdf.set_index('objectid').loc[compareid_select.value, 'datetime'] - gdf.set_index('objectid').loc[igniteid_select.value, 'datetime']
deltadt_label = Label(value=str(int(deltadt.total_seconds()/60)) + ' minutes')


vbox = VBox([igniteid_select, ignitedt_label, compareid_select, comparedt_label, deltadt_label])

igniteid_select.observe(partial(observe_objectid_slider, m, gdf, vbox), names='value')
compareid_select.observe(partial(observe_objectid_slider, m, gdf, vbox), names='value')

initiate_perimeters(gdf, igniteid_select.value, compareid_select.value)


widget_control = WidgetControl(widget = vbox, position='topleft')
m.add_control(widget_control) 

m.add_control(ZoomControl(position='topleft'))
m.add_control(ScaleControl(position='topleft'))

UIcontrol = WidgetControl(widget = usr.UI, position='topright')
m.add_control(UIcontrol)
m

In [ ]:
usr.UI

In [ ]:
usr.mainapi.run_farsite()

In [ ]:
 
def initiate_wktlayers(gdf, newws, newwd):
    add_mask = ((gdf['windspeed'] == newws) & 
                   (gdf['winddirection'] == newwd))
    
    gdf.loc[add_mask, 'WKTLayer'].apply(lambda wlayer: m.add_layer(wlayer))
    
def update_wktlayers(gdf, oldws, newws, oldwd, newwd):    
    remove_mask = ((gdf['windspeed'] == oldws) & 
                   (gdf['winddirection'] == oldwd))
    add_mask = ((gdf['windspeed'] == newws) & 
                   (gdf['winddirection'] == newwd))
    
    gdf.loc[remove_mask, 'WKTLayer'].apply(lambda wlayer: m.remove_layer(wlayer))
    gdf.loc[add_mask, 'WKTLayer'].apply(lambda wlayer: m.add_layer(wlayer))
    
def test_observe(m, gdf, vbox, event):
    if event['owner'].description == 'Wind Speed':
        oldws = event['old']
        newws = event['new']
        oldwd = vbox.children[1].value
        newwd = vbox.children[1].value
    elif event['owner'].description == 'Wind Direction':
        oldws = vbox.children[0].value
        newws = vbox.children[0].value
        oldwd = event['old']
        newwd = event['new']
        
    update_wktlayers(gdf, oldws, newws, oldwd, newwd)
    
gdf = usr.mainapi.db.gdfsimulation.to_crs(epsg=4326)
gdf['WKTLayer'] = gdf.apply(lambda row: WKTLayer(wkt_string = row['geometry'].wkt), axis=1)

centerlat = 34.31984085482566
centerlon = -119.03909139010659


m = Map(
    basemap=basemaps.Esri.WorldTopoMap,
    center=(centerlat, centerlon),
    zoom=10,
    layout=Layout(height='700px')
)

m.add_control(ScaleControl())




ws = gdf['windspeed'].unique()
wd = gdf['winddirection'].unique()
dt = gdf['datetime'].unique()

ws_select = SelectionSlider(description='Wind Speed', options=ws)
wd_select = SelectionSlider(description='Wind Direction', options=wd)

vbox = VBox([ws_select, wd_select])

ws_select.observe(partial(test_observe, m, gdf, vbox), names='value')
wd_select.observe(partial(test_observe, m, gdf, vbox), names='value')

widget_control = WidgetControl(widget = vbox)
m.add_control(widget_control)    
    
initiate_wktlayers(gdf, vbox.children[0].value, vbox.children[1].value)

m

In [ ]:
gdf = usr.mainapi.db.gdfsimulation.to_crs(epsg=4326)
gdf['WKTLayer'] = gdf.apply(lambda row: WKTLayer(wkt_string = row['geometry'].wkt), axis=1)
# gdf['WKTLayer_visibility'] = False

centerlon = gdf.iloc[0]['geometry'].centroid.x
centerlat = gdf.iloc[0]['geometry'].centroid.y

m = Map(
    basemap=basemaps.Esri.WorldTopoMap,
    center=(centerlat, centerlon),
    zoom=10,
    layout=Layout(height='700px')
)

m.add_control(ScaleControl())
    


m

In [ ]:
m

In [ ]:

# ## Extract the JSON object from the data returned on our GET request

# rTemperature = r_json['features'][0]['properties']['temperature']
# rTime = r_json['features'][0]['properties']['timestamp']

# rTimeMins = []
# for i, val in enumerate(rTime):
# 	rTimeMins.append( (arrow.get(rTime[i]).timestamp - arrow.get(rTime[0]).timestamp) / 60 / 60 )

# plt.plot(rTimeMins,rTemperature,'-og',label='Actual')
# plt.xlabel("time (hours)")
# plt.ylabel("temperature (F)")
# minFive = int(min(rTemperature)) - (int(min(rTemperature))%5)
# maxFive = (int(max(rTemperature)) +5) - ( (int(max(rTemperature)) +5) % 5 )
# plt.yticks(np.arange(minFive, maxFive+1, (maxFive - minFive)/2))
# plt.gca().yaxis.grid(True)
# title = "Temperature at Alert Airport, NU\n(Past 24 Hours)"
# plt.title(title)
# plt.show()

In [ ]:
r_json

### Create Landscape file

In [ ]:
gdf_5070 = usr.mainapi.db.gdfignition.set_crs(epsg=5070)
gdf_4326 = gdf_5070.to_crs(epsg=4326)
geom_5070 = gdf_5070.iloc[0]['geometry']
geom = gdf_4326.iloc[0]['geometry']

In [ ]:
%%time

bounds = geom_5070.bounds

ulx = bounds[0]-10000
uly = bounds[3]+10000
lrx = bounds[2]+10000
lry=  bounds[1]-10000

fname_lst = {'density': 'US_140CBD_12052016/Grid/us_140cbd', 
             'base': 'US_140CBH_12052016/Grid/us_140cbh', 
             'cover': 'US_140CC_12052016/Grid/us_140cc', 
             'height': 'US_140CH_12052016/Grid/us_140ch', 
             'fuel': 'US_140FBFM40_20180618/Grid/us_140fbfm40', 
             'aspect': 'Aspect/Grid/us_asp', 
             'elevation': 'DEM_Elevation/Grid/us_dem', 
             'slope': 'Slope/Grid/us_slp'}
type_lst = {'density': 'cbd',
            'base': 'cbh',
            'cover': 'cc',
            'height': 'ch',
            'fuel': 'fuel',   # fbfm40
            'aspect': 'aspect',
            'elevation': 'elevation', #dem
            'slope': 'slope'}

from_folder = os.path.join('/data', 'firemap', 'landfire', 'mosaic')
to_folder = '/home/tcaglar/farsite/inputs/landscapes/'

# Create the asc files
ascpath_lst = {}
for (key, fname) in fname_lst.items():
    ascpath_lst[key] = f'{os.path.join(to_folder, "test")}-{key}.asc'
    os.system(f'gdal_translate -of AAIGrid -a_nodata -32768 -projwin {ulx} {uly} {lrx} {lry} {os.path.join(from_folder, fname)} {ascpath_lst[key]}')
    
# # Extra for the tif file
# os.system(f'gdal_translate -of GTiff -a_nodata -32768 -projwin {ulx} {uly} {lrx} {lry} {os.path.join(from_folder, fname_lst["elevation"])} {os.path.join(to_folder, "test")}-elevation.tif')


In [ ]:
lcppath = f'{os.path.join(to_folder, "test")}-maria'
# ascpath_lst

In [ ]:
lcpmakepath = '/home/tcaglar/farsite/src/lcpmake'

base_command = f'{lcpmakepath} -latitude {m.center[0]} -landscape {lcppath}'
run_command = base_command
for (key, ascpath) in ascpath_lst.items():
    run_command += f' -{key} {ascpath}'
    
os.system(run_command)

In [ ]:
ascpath_lst

In [ ]:
m = Map(center = (geom.centroid.y, geom.centroid.x))
m.add_layer(WKTLayer(wkt_string = geom.wkt))
m

In [ ]:
def printSomething(text):
    print('Inside print')
    return text
    
def callbackFunction(value):
    A.append(value)
A = []


pool = Pool(processes=4)
for ix in range(126):
    pool.apply_async(partial(printSomething, ix), callback=callbackFunction)
pool.close()
pool.join()

In [ ]:
A

In [ ]:
count = 0.0
for (runfile, value) in usr.mainapi.runfile_done.items():
    count += value

usr.mainapi.loading_widget.value = 0

In [ ]:
usr.mainapi.update_loading()

In [ ]:
usr.mainapi.runfile_done[runfile] = 1

In [ ]:
usr.mainapi.farsite_lst[0].runfile.tofile()

In [ ]:
dt = {usr.mainapi.farsite_lst[0].runfile: 'test'}
dt[usr.mainapi.farsite_lst[0].runfile]

In [ ]:
usr.mainapi.runfile_lst[0].tofile()

In [ ]:
pd.read_pickle('/home/tcaglar/data/test_table.pkl')

In [ ]:
usr.UI

In [ ]:
gdf = gpd.read_file('/mnt/c/Users/tolk1/Dropbox/SDSC/SDGE/data/Lidar/Documents/LiDAR/Distribution/C11/2017_12_11/GIS-Data/C11_Data.gdb/')

In [ ]:
m = Map(
    basemap=basemaps.Esri.WorldTopoMap,
    center=(32.7157, -117.1611),
    zoom=10,
    layout=Layout(height='700px')
)

m.add_control(ScaleControl())
m.add_layer(g)

In [ ]:
usr.db.gdfsimulation

In [ ]:
usr.mainapi.farsite_lst[0].runfile.configfile.FARSITE_TIMESTEP

In [ ]:
gdf = gpd.read_file(mainapi.farsite_lst[2].runfile.outpath + '_Perimeters.shp')
minutes = gdf['Elapsed_Mi'].unique()

gdf0 = gdf[gdf['Elapsed_Mi'] == minutes[0]]
polygon_lst = [Polygon(value) for value in gdf0['geometry'].values]
multipoly = MultiPolygon()
for poly in polygon_lst:
    multipoly = multipoly.union(poly.buffer(0))
    
multipoly


In [ ]:
crs = gpd.read_file(mainapi.farsite_lst[0].runfile.outpath + '_Perimeters.shp').crs

In [ ]:
geom = gpd.read_file(mainapi.farsite_lst[0].runfile.outpath + '_Perimeters.shp').loc[0, 'geometry']
mainapi.db.gdfsimulation.loc['db8ac00b671441fbbd5f86e277f1b0d3', 'igniteidx'] = 'test'
gs = gpd.GeoSeries(index=['db8ac00b671441fbbd5f86e277f1b0d3'], data=geom)
mainapi.db.gdfsimulation.loc['db8ac00b671441fbbd5f86e277f1b0d3', 'geometry'] = gs

In [ ]:
gdf = gpd.read_file(mainapi.dftable.iloc[0]['filepath'])
gdf.crs

In [ ]:
gdf['geometry'][0]

In [ ]:
gdftable = gpd.GeoDataFrame(dftable)
gdftable.loc['f23196b034474744bdca7df94b13e0f7', 'geometry'] = gdf['geometry'][0]
gdftable

In [ ]:
Exception

In [ ]:
db = Database(fp)

In [ ]:
fpath = mainapi.dftable.loc['3c01d3099b094139873113e19c9f785a', 'filepath']
# geoms = gpd.read_file(fpath)['geometry']
# for geom in geoms:
#     display(geom)
#     break
    
gdf = gpd.read_file(fpath)
row = gdf.iloc[0]
geom = row['geometry']
gdf

In [ ]:
# Setup the database for reading
dftable = pd.read_pickle(fp.dfpath)

# Collect the tables in dataframe format
# Table 1 - ignition



# Table 2 - barrier
gdfbarrier = gpd.GeoDataFrame(dftable[dftable['filetype'] == 'Barrier'])
for (idx, ignition) in gdfbarrier.iterrows():
    geom = gpd.read_file(ignition['filepath']).loc[0,'geometry']
    gdfbarrier.loc[idx, 'shape'] = geom.to_wkb()

gs = gpd.GeoSeries.from_wkb(gdfignition['shape'])
gdfbarrier['geometry'] = gs
gdfbarrier.drop(columns='shape')

# Table 3 - landscape
# Table 4 - simulation

In [ ]:
gdfignition.loc['714f0cf8d475462a97c651642a3d2525', 'shape']

In [ ]:
# .from_wkb()

gdfignition

### Set the pickle file for maria fire

In [ ]:
# Read measured fire perimeter and convert to EPSG=5070
gdf_maria = gpd.read_file('/home/tcaglar/data/maria-wgs84/')

# ignition geodataframe
gdf_ignition = gdf_maria.to_crs(epsg=5070)
# reindex using the objectid
gdf_ignition.set_index('objectid', inplace=True)

# Remove the objectids 21230 and 21233
gdf_ignition.drop(index=[21230, 21233], inplace=True)

# Calculate datetime and add to gdf
gdf_ignition['datetime'] = pd.to_datetime(gdf_ignition['YYYYMMDD'].astype(str) + ':' + gdf_ignition['24_HHMMSS'], 
               format='%Y%m%d:%H:%M:%S')

In [ ]:
# igniteids = gdf_ignition.index
dftable = pd.DataFrame(columns=['filetype', 'objectid', 'filepath', 'datetime'])
for (objectid, row) in gdf_ignition.iterrows():
    igniteidx = uuid.uuid4().hex
    dftable.loc[igniteidx, 'filepath'] = os.path.join('/home/tcaglar/farsite/inputs/maria_ignite','maria_'+str(objectid)+'.shp')
    dftable.loc[igniteidx, 'filetype'] = 'Ignition'
    dftable.loc[igniteidx, 'objectid'] = objectid
    dftable.loc[igniteidx, 'datetime'] = row['datetime']

lcpidx = uuid.uuid4().hex
dftable.loc[lcpidx, 'filepath'] = '/home/tcaglar/farsite/inputs/landscapes/mariafire.lcp'
dftable.loc[lcpidx, 'filetype'] = 'Landscape'

barrieridx = uuid.uuid4().hex
dftable.loc[barrieridx, 'filepath'] = '/home/tcaglar/farsite/inputs/barriers/NoBarrier/NoBarrier.shp'
dftable.loc[barrieridx, 'filetype'] = 'Barrier'

# dftable.to_pickle('/home/tcaglar/data/test_table.pkl')
dftable = pd.read_pickle('/home/tcaglar/data/test_table.pkl')

In [ ]:
dftable